## Modelo de entrenamiento para la base de datos golf

#### Importación de librerías

Importamos las librerías pandas y numpy para agilizar el manejo de la base de datos.

In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats

#### Definición de variables

##### traform_data(df)

Esta función es la encargada de transformar el `DataFrame` *`df`*. La transformación consiste en reemplazar la columna `outlook`, que contiene los valores ***sunny***, ***rainy***, y ***outcast***, por tres nuevas columnas con los mismos nombres. Los valores de estas columnas son un $1$ o un $0$, dependiendo del antiguo valor de `outlook`.  

> **NOTA:** La transformación de la base de datos ha sido programada específicamente para la base de datos `golf` y cualquier otra base de datos con la misma estructura.  
> En caso de introducir una base de datos que no cumpla esta condición, el programa no arrojará los resultados esperados.  


In [113]:
def transform_data(df):
    dummies       = pd.get_dummies(df['outlook'])
    df            = pd.concat([df, dummies], axis=1)
    df            = df.drop(['outlook'], axis=1)
    cols          = df.columns.tolist()
    cols          = cols[-1 : -4 : -1] + cols[:-3]
    df[cols[:-1]] = df[cols[:-1]].astype('int64')
    return df[cols]

##### read_data(path)

Una simple función que lee una base de datos de un archivo csv desde el `path` a un `DataFrame`, y retorna la transformación de este.

In [114]:
def read_data(path):
    df = pd.read_csv(path)
    return transform_data(df)

##### split_data(df, train_size)

Función encargada de separar las filas que usaremos para el entrenamiento del modelo de las filas que usaremos para probar el modelo.  

Se recibe el `DataFrame`, y el tamaño de la muestra de entrenamiento. Seguido creamos una variable `sample_df` que almacena una muestra de `df` pero organizado de forma aleatoria por el método `df.sample(frac = 1)` .  

> **NOTA:** El método `sample()` se encarga de crear y retornar un `DataFrame` muestra, donde las filas han sido organizadas aleatoriamente. El *keyarg* `frac` representa el porcentaje de `df` que servirá como muestra. En esta ocasión está igualado a $1$ ya que necesitamos todo el `DataFrame`.

Almacenamos los valores desde $0$ hasta `train_size-1` de esta muestra en la variable `train`. El resto es almacenado en la variable `test`.  

Se retorna un `tuple` con ambos `DataFrames`.

In [115]:
def split_data(df, train_size):
    sample_df = df.sample(frac=1)
    train     = sample_df[ : train_size]
    test      = sample_df[train_size : ]
    return train, test

##### euclidean_distance(p, q)

Función encargada de calcular la distancia euclidiana entre la fila test $p$ y la fila train $q$.  

Se reciben dos `numpy arrays` y se excluye el último valor de ambos, el cual representa el valor de `play`.
Dado que son `numpy array` los arreglos pueden ser tratados como vectores euclidianos, por lo que hacemos la sustracción, y la elevación al cuadrado de forma directa. 

Esta operación retorna un nuevo `numpy array`, el cual es pasado como parámetro a la función `np.sum()`. `np.sum()` retorna la suma de todos los valores que se encuentran dentro de el *array*. El resultado de esta suma es pasado a la función `np.sqrt()` para calcular su raíz cuadrada.  

Finalmente, el valor de la raíz cuadrada es retornado.

In [116]:
def euclidean_distance(p, q):
    x = np.array(p.values)[:-1]
    y = np.array(q.values)[:-1]
    return np.sqrt(np.sum((y - x) ** 2))

##### kNearestNeighbors(x, train_data, k)

Función encargada de retornar los vecinos más cercanos a $p$.  

Se recibe los siguientes argumentos:
- `p` : Un `numpy array` que contiene la información de la fila a evaluar 
- `k` : El número de vecinos a tomar en cuenta 
- `train_data` : Un `DataFrame` con las filas de entrenamiento.  
  
Se crea un arreglo `neighbors` usando un técnica llamada *list comprehension*. El tamaño del arreglo es igual al tamaño de `train_data`.   
Esta variable almacena un arreglo de *tuples*, los cuales tienen como primer elemento la distancia entre $p$ y $q_i$. El segundo elemento de cada `tuple` es el índice de $q_i$.  

El arreglo se organiza de forma ascendente basándose en los valores de las distancias euclidianas y finalmente se retornan los primeros $k$ elementos del arreglo, los cuales representan los $k$ vecinos más cercanos.

In [117]:
def kNearestNeighbors(p, train_data, k):
    neighbors = [(euclidean_distance(p, q), i) for i, q in train_data.iterrows()]    
    neighbors.sort()
    return neighbors[ :k]

##### train(k, train_data, test_data)

Función encargada de predecir el valor de `play` de cada una de las filas en `test_data`, y retornar el porcentaje de error del modelo.  

Recibe los siguientes parámetros:
- `k` : Número de vecinos a tomar en cuenta
- `train_data` : Un `DataFrame` con las filas de entrenamiento.
- `test_data` : Un `DataFrame` con las filas de prueba.  

Se inicializa una variable `errors` con un valor de $0$ que representa el número de resultados erróneos. Se aumenta en uno cada que la predicción sea distinta al valor de `p['play']`.  
Una vez terminadas las predicciones, se retorna el porcentaje de errores en el modelo. 

In [118]:
def train(k, train_data, test_data):
    errors = 0
    for i, p in test_data.iterrows():
        y          = kNearestNeighbors(p, train_data, k)
        results    = df.iloc[[tup[1] for tup in y]]['play'].tolist()
        prediction = stats.mode(results)

        # print(f'Punto: {p.values, i} \nVecinos: {y} \nPredicción: {prediction}\n\n')

        if prediction != p['play']:
            errors += 1
        
    return errors / len(test_data)

In [119]:
df = read_data('golf.csv')
train_data, test_data = split_data(df, 10)
print(f'{train_data}\n\n')
print(test_data)

    sunny  rainy  overcast  temperature  humidity  windy play
11      0      0         1           72        90      1  yes
13      0      1         0           71        91      1   no
10      1      0         0           75        70      1  yes
5       0      1         0           65        70      1   no
3       0      1         0           70        96      0  yes
4       0      1         0           68        80      0  yes
7       1      0         0           72        95      0   no
0       1      0         0           85        85      0   no
6       0      0         1           64        65      1  yes
12      0      0         1           81        75      0  yes


   sunny  rainy  overcast  temperature  humidity  windy play
9      0      1         0           75        80      0  yes
1      1      0         0           80        90      1   no
2      0      0         1           83        86      0  yes
8      1      0         0           69        70      0  yes


In [120]:
train(3, train_data, test_data)

0.0

In [121]:
train(5, train_data, test_data)

0.25

In [122]:
train(7, train_data, test_data)

0.25